## imports and config

In [1]:
!pip install pyserini==0.9.4.0
!pip install jsonlines

     |████████████████████████████████| 60.4MB 68kB/s 
     |████████████████████████████████| 1.1MB 48.5MB/s 


In [2]:
import os
import json 
import random

from tqdm import tqdm
import xml.etree.ElementTree as ET
import jsonlines 
from pyserini.search import SimpleSearcher

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

%cd 'drive/My Drive/CS646_Final_Project/'

!ls

Mounted at /content/drive/
/content/drive/My Drive/CS646_Final_Project
'Baseline BERT-ADA'	       Data			     Siamese
 BM25			      'Linear Interpolation Merge'
 cs646_milestone2_steps.gdoc  'RM3 Merge'


In [43]:
!pwd

/content/drive/My Drive/CS646_Final_Project


In [ ]:
!ls Siamese_results/sbert_bert_ada_joint_online_contrastive/top_1000/query_results_restaurants_sbert_bert_ada_joint_online_contrastive.txt

query_results_laptop_sbert_bert_ada_joint_online_contrastive.txt
query_results_restaurants_sbert_bert_ada_joint_online_contrastive.txt
trec_eval_output.txt


In [46]:
siamese_result_restaurant_path = os.path.join('Siamese', 'Siamese_results', 'sbert_bert_ada_joint_online_contrastive', 'top_1000', 'query_results_restaurants_sbert_bert_ada_joint_online_contrastive.txt')
siamese_result_laptop_path = os.path.join('Siamese', 'Siamese_results', 'sbert_bert_ada_joint_online_contrastive', 'top_1000', 'query_results_laptop_sbert_bert_ada_joint_online_contrastive.txt')

bm25_result_restaurant_path = os.path.join('./', 'BM25', 'test_query_results_restaurant.txt')
bm25_result_laptop_path = os.path.join('./', 'BM25', 'test_query_results_laptop.txt')

merge_laptop_result_path = os.path.join('RM3_Merge', 'rm3_merged_results', 'query_results_merged_laptop.txt')
merge_restaurant_result_path = os.path.join('RM3_Merge', 'rm3_merged_results', 'query_results_merged_restaurant.txt')

avg_laptop_result_path = os.path.join('RM3 Merge', 'rm3_merged_results', 'query_results_avg_laptop.txt')
avg_restaurant_result_path = os.path.join('RM3 Merge', 'rm3_merged_results', 'query_results_avg_restaurant.txt')

## combine result lists

In [85]:
merged_query_docs_rest = {}
merged_query_docs_lap = {}

In [86]:
with open(bm25_result_restaurant_path) as f:
    for line in f:
        line = line.split(' ')
        qid = line[0]
        docid = line[2]
        score = float(line[4])
        if qid not in merged_query_docs_rest:
          merged_query_docs_rest[qid] = {
              'docid': [],
              'score': []
          }
        if docid not in merged_query_docs_rest[qid]['docid']:
          merged_query_docs_rest[qid]['docid'].append(docid)
          merged_query_docs_rest[qid]['score'].append(score)

# normalize bm25 
for qid in merged_query_docs_rest.keys():
  max_ = max(merged_query_docs_rest[qid]['score'])
  min_ = min(merged_query_docs_rest[qid]['score'])
  if len(merged_query_docs_rest[qid]['score']) <= 1:
    merged_query_docs_rest[qid]['score'][0] = 0.5
    continue
  for i,score in enumerate(merged_query_docs_rest[qid]['score']):
    merged_query_docs_rest[qid]['score'][i] = str((score-min_)/(max_- min_))

with open(siamese_result_restaurant_path) as f:
    for line in f:
        line = line.split(' ')
        qid = line[0]
        docid = line[2]
        score = line[4]
        if qid not in merged_query_docs_rest:
          merged_query_docs_rest[qid] = {
              'docid': [],
              'score': []
          }
        if docid not in merged_query_docs_rest[qid]['docid']:
          merged_query_docs_rest[qid]['docid'].append(docid)
          merged_query_docs_rest[qid]['score'].append(score)
        else:
          ind = merged_query_docs_rest[qid]['docid'].index(docid)
          prev_score = float(merged_query_docs_rest[qid]['score'][ind])
          merged_query_docs_rest[qid]['score'][ind] = str((prev_score + float(score)) / 2)

with open(bm25_result_laptop_path) as f:
    for line in f:
        line = line.split(' ')
        qid = line[0]
        docid = line[2]
        score = float(line[4])
        if qid not in merged_query_docs_lap:
          merged_query_docs_lap[qid] = {
              'docid': [],
              'score': []
          }
        if docid not in merged_query_docs_lap[qid]['docid']:
          merged_query_docs_lap[qid]['docid'].append(docid)
          merged_query_docs_lap[qid]['score'].append(score)

# normalize bm25 
for qid in merged_query_docs_lap.keys():
  max_ = max(merged_query_docs_lap[qid]['score'])
  min_ = min(merged_query_docs_lap[qid]['score'])
  if len(merged_query_docs_lap[qid]['score']) <= 1:
    merged_query_docs_lap[qid]['score'][0] = 1
    continue
  for i,score in enumerate(merged_query_docs_lap[qid]['score']):
    merged_query_docs_lap[qid]['score'][i] = str((score-min_)/(max_- min_))

with open(siamese_result_laptop_path) as f:
    for line in f:
        line = line.split(' ')
        qid = line[0]
        docid = line[2]
        score = line[4]
        if qid not in merged_query_docs_lap:
          merged_query_docs_lap[qid] = {
              'docid': [],
              'score': []
          }
        if docid not in merged_query_docs_lap[qid]['docid']:
          merged_query_docs_lap[qid]['docid'].append(docid)
          merged_query_docs_lap[qid]['score'].append(score)
        else:
          ind = merged_query_docs_lap[qid]['docid'].index(docid)
          prev_score = float(merged_query_docs_lap[qid]['score'][ind])
          merged_query_docs_lap[qid]['score'][ind] = str((prev_score + float(score)) / 2)


In [62]:
print(lap_min)
print(lap_max)
print(rest_min)
print(rest_max)

0.0
1.0
0.0
1.0


In [63]:
print(si_lap_min)
print(si_lap_max)
print(si_rest_min)
print(si_rest_max)

0.34499401
0.97061199
0.34683552
0.97867191


In [87]:
!rm ./RM3\ Merge/rm3_merged_results/query_results_avg_laptop.txt

for i,qid in tqdm(enumerate(merged_query_docs_lap.keys())):
  docs = merged_query_docs_lap[qid]['docid']
  scores = merged_query_docs_lap[qid]['score']
  doc_sorted = sorted(zip(docs,scores), key=lambda x: x[1], reverse=True)

  for j,doc in enumerate(doc_sorted):
    docid = doc[0]
    score = doc[1]

    line = str(i+1) + ' Q0 ' + docid + ' ' + str(j+1) + ' ' + str(score) + ' avg'

    with open(avg_laptop_result_path, 'a') as f:
      f.write("%s\n" % line)



0it [00:00, ?it/s]

1it [00:02,  2.84s/it]

2it [00:05,  2.88s/it]

3it [00:08,  2.84s/it]

4it [00:11,  2.79s/it]

5it [00:14,  2.80s/it]

6it [00:16,  2.76s/it]

7it [00:19,  2.78s/it]

8it [00:22,  2.97s/it]

9it [00:25,  2.88s/it]

10it [00:28,  2.83s/it]

11it [00:31,  2.85s/it]

12it [00:33,  2.80s/it]

13it [00:37,  2.98s/it]

14it [00:40,  2.98s/it]

15it [00:43,  2.90s/it]

16it [00:45,  2.84s/it]

17it [00:48,  2.78s/it]

18it [00:51,  2.85s/it]

19it [00:54,  2.83s/it]

20it [00:57,  3.02s/it]

21it [01:00,  2.90s/it]

22it [01:02,  2.82s/it]

23it [01:05,  2.77s/it]

24it [01:08,  2.74s/it]

25it [01:11,  2.80s/it]

26it [01:13,  2.76s/it]

27it [01:16,  2.74s/it]

28it [01:19,  2.71s/it]

29it [01:21,  2.70s/it]

30it [01:24,  2.77s/it]

31it [01:28,  3.16s/it]

32it [01:32,  3.16s/it]

33it [01:35,  3.36s/it]

34it [01:38,  3.18s/it]

35it [01:41,  3.03s/it]

36it [01:44,  3.02s/it]

37it [01:47,  2.96s/it]

38it [01:49,  2.91s/it]

39it [01:53,  3.04s/it]

40it [01:55,

In [88]:
!rm ./RM3\ Merge/rm3_merged_results/query_results_avg_restaurant.txt

for i,qid in tqdm(enumerate(merged_query_docs_rest.keys())):
  docs = merged_query_docs_rest[qid]['docid']
  scores = merged_query_docs_rest[qid]['score']
  doc_sorted = sorted(zip(docs,scores), key=lambda x: x[1], reverse=True)

  for j,doc in enumerate(doc_sorted):
    docid = doc[0]
    score = doc[1]

    line = str(i+1) + ' Q0 ' + docid + ' ' + str(j+1) + ' ' + str(score) + ' avg'

    with open(avg_restaurant_result_path, 'a') as f:
      f.write("%s\n" % line)



0it [00:00, ?it/s]

1it [00:02,  2.75s/it]

2it [00:05,  2.77s/it]

3it [00:08,  2.76s/it]

4it [00:10,  2.72s/it]

5it [00:13,  2.72s/it]

6it [00:16,  2.77s/it]

7it [00:20,  2.99s/it]

8it [00:23,  3.09s/it]

9it [00:26,  3.17s/it]

10it [00:29,  3.01s/it]

11it [00:32,  2.91s/it]

12it [00:34,  2.85s/it]

13it [00:37,  2.80s/it]

14it [00:40,  2.78s/it]

15it [00:43,  2.84s/it]

16it [00:45,  2.83s/it]

17it [00:49,  3.05s/it]

18it [00:52,  2.94s/it]

19it [00:54,  2.88s/it]

20it [00:58,  3.07s/it]

21it [01:01,  2.96s/it]

22it [01:03,  2.85s/it]

23it [01:06,  2.83s/it]

24it [01:09,  2.79s/it]

25it [01:11,  2.78s/it]

26it [01:14,  2.75s/it]

27it [01:17,  2.71s/it]

28it [01:20,  2.97s/it]

29it [01:23,  2.88s/it]

30it [01:26,  2.85s/it]

31it [01:28,  2.78s/it]

32it [01:31,  2.76s/it]

33it [01:34,  2.72s/it]

34it [01:37,  2.98s/it]

35it [01:40,  2.91s/it]

36it [01:43,  2.86s/it]

37it [01:46,  2.81s/it]

38it [01:48,  2.78s/it]

39it [01:51,  2.76s/it]

40it [01:55,

## create index for each query

In [ ]:
semeval_path = os.path.join('data', 'SemEval2014_Task4')
collection_path = os.path.join('data', 'query_collections')
index_path = os.path.join('/content/drive/My Drive/CS646_Final_Project', 'data', 'indexes', 'merged_indexes')

In [ ]:
files = [
  'Laptop_Train_v2.xml',
  'Laptops_Test_Gold.xml',
  'Restaurants_Test_Gold.xml',
  'Restaurants_Train_v2.xml'
]

new_files = {
   'Laptop_Train_v2.xml': 'laptop_train.jsonl',
   'Laptops_Test_Gold.xml': 'laptop_test.jsonl',
   'Restaurants_Test_Gold.xml': 'restaurant_test.jsonl',
   'Restaurants_Train_v2.xml': 'restaurant_train.jsonl'
}

In [ ]:
# laptops

for qid in tqdm(merged_query_docs_lap.keys()):
  cur_collection_path = os.path.join(collection_path, 'laptop', 'q{}_collection'.format(qid))
  os.mkdir(cur_collection_path)

  cur_index_path = os.path.join(index_path, 'laptop', 'q{}_index'.format(qid))
  os.mkdir(cur_index_path)

  file_id = 0
  for file in files:
    filepath = os.path.join(semeval_path, file)
    savepath = os.path.join(cur_collection_path, new_files[file])
    #!rm $savepath
    #print(filepath)
    file_id += 1 

    with open(filepath) as semeval_file:
      sentence_elements = ET.parse(semeval_file).getroot().iter('sentence')

      for id_, s in enumerate(sentence_elements):

        # only add doc to index if in merged result list
        docid = 'doc' + str(file_id) + str(id_)
        if docid not in merged_query_docs_lap[qid]:
          continue

        sent = s.find('text').text

        aspects = []
        for o in s.iter('aspectTerm'):
          aspect = o.get('term')
          sentiment = o.get('polarity')

          if sentiment != 'conflict': 
            aspects.append((aspect,sentiment))

        doc = {
            'id': docid,
            'contents': sent,
            #'aspects': aspects 
        }

        with jsonlines.open(savepath, mode='a') as writer:
          writer.write(doc)

  !python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 1 -input $cur_collection_path \
 -index $cur_index_path -storePositions -storeDocvectors -storeRaw

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# restaurant

for qid in tqdm(merged_query_docs_rest.keys()):
  cur_collection_path = os.path.join(collection_path, 'restaurant', 'q{}_collection'.format(qid))
  os.mkdir(cur_collection_path)

  cur_index_path = os.path.join(index_path, 'restaurant', 'q{}_index'.format(qid))
  os.mkdir(cur_index_path)

  file_id = 0
  for file in files:
    filepath = os.path.join(semeval_path, file)
    savepath = os.path.join(cur_collection_path, new_files[file])
    #!rm $savepath
    #print(filepath)
    file_id += 1 

    with open(filepath) as semeval_file:
      sentence_elements = ET.parse(semeval_file).getroot().iter('sentence')

      for id_, s in enumerate(sentence_elements):

        # only add doc to index if in merged result list
        docid = 'doc' + str(file_id) + str(id_)
        if docid not in merged_query_docs_rest[qid]:
          continue

        sent = s.find('text').text

        aspects = []
        for o in s.iter('aspectTerm'):
          aspect = o.get('term')
          sentiment = o.get('polarity')

          if sentiment != 'conflict': 
            aspects.append((aspect,sentiment))

        doc = {
            'id': docid,
            'contents': sent,
            #'aspects': aspects 
        }

        with jsonlines.open(savepath, mode='a') as writer:
          writer.write(doc)

  !python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 1 -input $cur_collection_path \
 -index $cur_index_path -storePositions -storeDocvectors -storeRaw

Output hidden; open in https://colab.research.google.com to view.

## run rm3 for each query

In [ ]:
result_laptop_path = os.path.join('RM3_Merge', 'rm3_merged_results', 'query_rm3_results_laptop.txt')
result_restaurant_path = os.path.join('RM3_Merge', 'rm3_merged_results', 'query_rm3_results_restaurant.txt')

laptop_queries_path = os.path.join('BM25', 'test_queries_laptop.txt')
restaurant_queries_path = os.path.join('BM25', 'test_queries_restaurant.txt')

In [ ]:
# laptop 
# remove if exist because appending
!rm $result_laptop_path

with open(laptop_queries_path) as f:
  q_num = 0
  for query in tqdm(f):
    q_num += 1
    cur_index_path = os.path.join(index_path, 'laptop', 'q{}_index'.format(q_num))
    searcher = SimpleSearcher(cur_index_path)
    searcher.set_rm3(10, 10, 0.5)
    hits = searcher.search(q=query, k=1000)

    for i in range(len(hits)):
      line = str(q_num) + ' Q0 ' + hits[i].docid + ' ' + str(i+1) + ' ' + '%.8f' % hits[i].score + ' rm3'
      
      with open(result_laptop_path, 'a') as f:
        f.write("%s\n" % line)

rm: cannot remove 'RM3_Merge/rm3_merged_results/query_rm3_results_laptop.txt': No such file or directory


475it [1:38:07, 12.39s/it]


In [ ]:
# restaurant 
# remove if exist because appending
!rm $result_restaurant_path

with open(restaurant_queries_path) as f:
  q_num = 0
  for query in f:
    q_num += 1
    cur_index_path = os.path.join(index_path, 'restaurant', 'q{}_index'.format(q_num))
    searcher = SimpleSearcher(cur_index_path)
    searcher.set_rm3(10, 10, 0.5)
    hits = searcher.search(q=query, k=1000)

    for i in range(len(hits)):
      line = str(q_num) + ' Q0 ' + hits[i].docid + ' ' + str(i+1) + ' ' + '%.8f' % hits[i].score + ' rm3'
      
      with open(result_restaurant_path, 'a') as f:
        f.write("%s\n" % line)

rm: cannot remove 'RM3_Merge/rm3_merged_results/query_rm3_results_restaurant.txt': No such file or directory
